<a href="https://colab.research.google.com/github/djv1234/NeurodegenerativeResearch/blob/main/AxonClassificationColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Input the files from Dr.B
from google.colab import files

uploaded = files.upload()


Saving pics for Dmitry.zip to pics for Dmitry.zip


In [ ]:
import zipfile
import os

zip_path = "pics for Dmitry.zip"  #make zip path
extract_dir = "/content/research_images"

os.makedirs(extract_dir, exist_ok=True)

# Extract pictures from zip
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("✅ Extracted files:")
print(os.listdir(extract_dir))

✅ Extracted files:
['Markers_Counter Window - 7700_tibial30hcut_100xOC_tilescan_01.tif', 'CellCounter_7700_tibial30hcut_100xOC_tilescan_01.xml', 'Markers_Counter Window - 7724-3_tibial30hcut_100xOC_tilescan_01.tif', '7708_tibial30hcut_100xOC_tilescan_01.tif', 'pilot.lif', '7724-3_tibial30hcut_100xOC_tilescan_01.tif', '7700_tibial30hcut_100xOC_tilescan_01.tif', '7720_tibial30hcut_100xOC_tilescan_01.tif', 'CellCounter_7708_tibial30hcut_100xOC_tilescan_01.xml', 'CellCounter_7720_tibial30hcut_100xOC_tilescan_01.xml', 'Markers_Counter Window - 7708_tibial30hcut_100xOC_tilescan_01.tif', 'CellCounter_7724-3_tibial30hcut_100xOC_tilescan_01.xml', 'Markers_Counter Window - 7720_tibial30hcut_100xOC_tilescan_01.tif']


In [ ]:
# Installed required libraries
!pip install -U pip
!pip install ultralytics opencv-python-headless pandas PyYAML

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 63.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 50.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [ultralytics]


In [ ]:
import torch
print("torch version:", torch.__version__)
print("GPU available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Device name:", torch.cuda.get_device_name(0))

torch version: 2.8.0+cu126
GPU available: True
Device name: Tesla T4


In [ ]:
import cv2, os, random, math
import numpy as np
root = "/content/axon_dataset"
os.makedirs(root+"/train/images", exist_ok=True)
os.makedirs(root+"/train/labels", exist_ok=True)
os.makedirs(root+"/val/images", exist_ok=True)
os.makedirs(root+"/val/labels", exist_ok=True)

def save_synth(path_img, path_lbl, label):
    h,w=256,256
    img = np.ones((h,w,3), dtype=np.uint8)*255
    cx = random.randint(40,w-40); cy = random.randint(40,h-40)
    if label==0: # intact: dark circle + 2 small dark dots (mitochondria)
        cv2.circle(img, (cx,cy), random.randint(12,22), (30,30,30), -1)
        # two mitochondria
        cv2.circle(img, (cx-6,cy), 2, (10,10,10), -1)
        cv2.circle(img, (cx+6,cy), 2, (10,10,10), -1)
    else: # degenerated: bean shape / bright with collapse
        pts = np.array([[cx-18,cy], [cx-8,cy-18],[cx+18,cy-6],[cx+12,cy+12]], np.int32)
        cv2.fillPoly(img,[pts], (200,200,200))
        cv2.circle(img, (cx,cy), 6, (30,30,30), -1) # inner dark mass sometimes
    cv2.imwrite(path_img, img)
    # create YOLO label (simple bbox around object)
    x1,y1,x2,y2 = cx-30,cy-30,cx+30,cy+30
    xcenter = ((x1+x2)/2)/w; ycenter=((y1+y2)/2)/h
    bw = (x2-x1)/w; bh=(y2-y1)/h
    with open(path_lbl,'w') as f:
        f.write(f"{label} {xcenter:.6f} {ycenter:.6f} {bw:.6f} {bh:.6f}\n")

# make 30 train, 10 val
for i in range(30):
    lab = 0 if i%2==0 else 1
    save_synth(f"/content/axon_dataset/train/images/img_{i:03d}.png", f"/content/axon_dataset/train/labels/img_{i:03d}.txt", lab)
for i in range(10):
    lab = 0 if i%2==0 else 1
    save_synth(f"/content/axon_dataset/val/images/val_{i:03d}.png", f"/content/axon_dataset/val/labels/val_{i:03d}.txt", lab)

print("Synthetic dataset created.")

Synthetic dataset created.


In [ ]:
import os
from PIL import Image
# opening .tif files and segmenting them into 1024x1024 pixel PNGs
img = Image.open("")
tile_size = 1024
output_dir = "tiles"
os.makedirs(output_dir, exist_ok=True)

for i in range(0, img.width, tile_size):
    for j in range(0, img.height, tile_size):
        box = (i, j, min(i+tile_size, img.width), min(j+tile_size, img.height))
        tile = img.crop(box)
        tile.save(f"{output_dir}/axon_{i}_{j}.png")

In [ ]:
# .html parser to output the number of degenerated vs. intact axons
import xml.etree.ElementTree as ET

def count_axons_from_cellcounter(html_path):
    tree = ET.parse(html_path)
    root = tree.getroot()

    intact_count = 0
    degenerated_count = 0

    for marker_type in root.findall(".//Marker_Type"):
        type_id = marker_type.find("Type").text
        markers = marker_type.findall("Marker")

        if type_id == "3":  # intact axons
            intact_count += len(markers)
        elif type_id == "4":  # degenerated axons
            degenerated_count += len(markers)

    total = intact_count + degenerated_count
    ratio = degenerated_count / intact_count if intact_count > 0 else float("inf")

    return {
        "intact_count": intact_count,
        "degenerated_count": degenerated_count,
        "total": total,
        "degenerate_to_intact_ratio": ratio
    }

# Count up the number of degenerated, intact axons, and ratio
result = count_axons_from_cellcounter(r"/content/research_images/CellCounter_7700_tibial30hcut_100xOC_tilescan_01.xml")
print("✅ Results:")
print(f"  Intact axons:      {result['intact_count']}")
print(f"  Degenerated axons: {result['degenerated_count']}")
print(f"  Total axons:       {result['total']}")
print(f"  Degenerated/Intact Ratio: {result['degenerate_to_intact_ratio']:.3f}")

✅ Results:
  Intact axons:      168
  Degenerated axons: 1166
  Total axons:       1334
  Degenerated/Intact Ratio: 6.940
